## dependencies

In [4]:
import napari
from napari.layers import Image, Tracks
# from dask.array.image import imread
from iohub import open_ome_zarr
import dask.array as da
import pandas as pd
# import matplotlib.pyplot as plt
import os
from pathlib import Path
from tifffile import imread

import numpy as np

from ultrack.utils import labels_to_contours

## Loading data

In [5]:
#*************************************************
path = '<USER DEFINED PATH>/3_segmentation/membrane/membrane_masks.zarr'
path_raw = '<USER DEFINED PATH>/2_deconvolution_and_registration/deconvolved_and_registered.zarr'
Tmax = 500    
working_directory = '.'       

flag_closedGaps = ''

#*************************************************

#Load CellPose labels 
ds_cellpose = open_ome_zarr(path)
labels = da.from_zarr(ds_cellpose["0/2/0"].data)[:Tmax,0]

#Load raw 
ds_raw = open_ome_zarr(path_raw)  #ds_raw.print_tree()
raw_cyto = da.from_zarr(ds_raw['0/2/0'].data)[:Tmax,0]
raw_mem = da.from_zarr(ds_raw['0/2/0'].data)[:Tmax,1]


#Load Tracking results
tracking_files = []
for f in os.listdir(working_directory):
    if f.endswith("tracks" + flag_closedGaps + ".csv"):
        tracking_files.append(f)
if len(tracking_files)==1:
    print('saved_tracks loaded')
    f = tracking_files[0]
    saved_tracks = pd.read_csv(working_directory+f)
else:
    del saved_tracks
    print("Error: multile _tracks.csv files in dir")

segments = da.from_zarr(working_directory + 'segments' + flag_closedGaps + '/')
# graph = inv_tracks_df_forest(saved_tracks)
# config = load_config(working_directory+'config.toml')

saved_tracks loaded


In [ ]:
detection, contours = labels_to_contours(
    labels=labels,
    sigma=2,
)

## Napari viewer

In [6]:
#******************
t = 200
z_slice  = 31
scale = (0.25, 0.075, 0.075)
#******************

viewer = napari.Viewer()

# Cytoplasm
layer_cyto = viewer.add_image(raw_cyto,gamma=1,scale=scale,colormap='magenta',contrast_limits=(0.0, .13),name='raw_nucleus')
layer_cyto.opacity = 1
layer_cyto._auto_contrast_source
layer_cyto.refresh()
layer_cyto.visible = True

# Membrane
layer_mem = viewer.add_image(raw_mem, gamma=1.3,scale=scale,colormap='cyan',contrast_limits=(0, 0.28),name='raw_membrane')
layer_mem.opacity = 0.58
layer_mem.visible = True

# CellPose
layer_cp = viewer.add_labels(labels,scale=scale,opacity=1, name="cellpose")
layer_cp.visible = False

# Ultrack - foreground
layer_foreground = viewer.add_labels(detection,scale=scale, name="foreground",color={0:'black', 1:(1, 1, 1, 1)},opacity=1)
layer_foreground.visible = False

# Ultrack - contours
layer_contours = viewer.add_image(contours,contrast_limits=(0, 0.32),scale=scale, name="contours")
layer_contours.visible = False

# Ultrack segments
layer_seg = viewer.add_labels(segments,scale=scale, name="segments")
layer_seg.contour = 0   # 0 by default
layer_seg.opacity = 1
layer_seg.visible = False

# Ultrack tracks
pad = 12
z_min, z_max = z_slice - pad, z_slice  + pad
df = saved_tracks
filtered_df = df[
    ((df['t'] - t).abs() < 1) &
    (df['z'] >= z_min) & (df['z'] < z_max)
]

filtered_df = df[df["track_id"].isin(filtered_df["track_id"].unique())]

layer_tracks = viewer.add_tracks(
    filtered_df[['track_id','t','z', 'y', 'x']],
    scale=scale,
    name = "tracks",
    visible=True,
    tail_length = 120, 
    tail_width = 5, 
    colormap = 'turbo',
    # graph=graph,
)
layer_tracks.blending = 'minimum'

viewer.dims.set_current_step(0, t)
viewer.dims.set_current_step(1, z_slice)
viewer.scale_bar.visible = True

viewer.camera.center = (0, 34.3, 49)
viewer.camera.zoom = 22

viewer.window.resize(1400, 1000)


# layer_cyto.gamma = 0.25


## Saving screenshots

In [ ]:
FIG_DIR = Path('./500_frames/')

viewer.dims.set_current_step(0, t)
viewer.dims.set_current_step(1, z_slice)

layer_cyto.visible = True
layer_mem.visible = True
layer_cp.visible = False
layer_foreground.visible = False
layer_contours.visible = False
layer_seg.visible = False
layer_tracks.visible = False

viewer.screenshot(FIG_DIR / "neuromast_thumbnails_0_raw.png")

layer_cyto.visible = False
layer_mem.visible = False
layer_cp.visible = True
layer_foreground.visible = False
layer_contours.visible = False
layer_seg.visible = False
layer_tracks.visible = False

viewer.screenshot(FIG_DIR / "neuromast_thumbnails_1_cellpose.png")

layer_cyto.visible = False
layer_mem.visible = False
layer_cp.visible = False
layer_foreground.visible = True
layer_contours.visible = False
layer_seg.visible = False
layer_tracks.visible = False

viewer.screenshot(FIG_DIR / "neuromast_thumbnails_2_foreground.png")

layer_cyto.visible = False
layer_mem.visible = False
layer_cp.visible = False
layer_foreground.visible = False
layer_contours.visible = True
layer_seg.visible = False
layer_tracks.visible = False

viewer.screenshot(FIG_DIR / "neuromast_thumbnails_3_contours.png")

layer_cyto.visible = False
layer_mem.visible = False
layer_cp.visible = False
layer_foreground.visible = False
layer_contours.visible = False
layer_seg.visible = True
layer_tracks.visible = False

viewer.screenshot(FIG_DIR / "neuromast_thumbnails_4_segments.png")

layer_cyto.visible = False
layer_mem.visible = False
layer_cp.visible = False
layer_foreground.visible = False
layer_contours.visible = False
layer_seg.visible = False
layer_tracks.visible = True
viewer.dims.set_current_step(0, 55)

viewer.screenshot(FIG_DIR / "neuromast_thumbnails_5_tracks.png")
